# Set up env

In [ ]:
import carla
import numpy as np
from src.simulator import Simulator
from src.agent import NCPAgent
from src.model import Model

import torchvision
from IPython.display import display
import sys
import os
import torch

sys.path.append("CARLA_SIM/PythonAPI/carla/")
from agents.navigation.basic_agent import BasicAgent


# Data collection

In [ ]:
import carla
from random import random

random_rotation = 3 * random() - 1.5
mapping = {
    'Town01_opt': {
        'loc': carla.Location(x=176.589493, y=133.239151, z=0.300000), # 15 spawn point
        'rotation': carla.Rotation(0, 0 + random_rotation, 0)
    },
    'Town02_opt': {
        'loc': carla.Location(x=193.779999, y=142.190002, z=0.500000), # 15 spawn point
        'rotation': carla.Rotation(0, -90 + random_rotation, 0)
    },
    'Town03_opt': {
        # 'loc': carla.Location(x=-88.710991, y=-119.565231, z=0.275307), # 1 spawn point
        # 'rotation': carla.Rotation(0, 90 + random_rotation, 0)
        'loc': carla.Location(x=-0.710991, y=-120.565231, z=0.275307), # 1 spawn point
        'rotation': carla.Rotation(0, 90 + random_rotation, 0)
    },
    'Town04_opt': {
        'loc': carla.Location(405.320374, -48.450779, 0.281942), #222 spawn point
        'rotation': carla.Rotation(0, -90 + random_rotation, 0)
    },
}
print(random_rotation)

In [ ]:
simulator = Simulator(world_name='Town03_opt', dump_data=True, debug=True)

In [ ]:
import time

simulator.world.get_spectator()#.set_transform()
    # carla.Transform(
    #     location=carla.Location(x=398.7934265136719,
    #                             y=-56.03200912475586,
    #                             z=3.37939715385437)))

simulator.spawn_car_with_camera(
    rel_coordinates=carla.Location(x=1.2, z=1.9), # camera coords
    vehicle_coordinates=mapping[simulator.world_name]['loc'],
    vehicle_rotation=mapping[simulator.world_name]['rotation'],
    image_param=(640,640)
)
vehicle = simulator.get_vehicle()



output_size = 4
units = 19
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

base_model = Model(output_size, units)
# ncp.to(device)
if not os.path.isdir(f'out/{simulator.world_name}'):
    os.mkdir(f'out/{simulator.world_name}')
with open(f'out/{simulator.world_name}/data.txt', 'a+') as f:
    f.write(f'timestamp start = {time.time()}\n')
agent = NCPAgent(simulator, base_model, target_speed=10)

next_waypoint  = [simulator.world.get_map().get_waypoint(vehicle.get_location(),
                                                    project_to_road=True,
                                                    lane_type=(carla.LaneType.Driving))]

waypoints = []
dist_between_waypoints = 15
waypoint_num = 50
# waypoint_num = 350
# waypoint_num = 100
# waypoint_num = 10
for _ in range(waypoint_num):
    waypoints.append(next_waypoint[-1])
    # simulator.world.get_spectator().set_transform(next_waypoint[-1].transform)
    next_waypoint = next_waypoint[-1].next(dist_between_waypoints)

dest_idx = 2
dest = waypoints[dest_idx].transform.location
agent.set_destination(dest)
agent.set_target_speed(10)
agent.ignore_traffic_lights(active=True)
agent.ignore_stop_signs(active=True)

In [ ]:
# # os.rmdir('./out/')

idx = 1
tmp = 0
waypoint = waypoints[idx]
to_PIL = torchvision.transforms.ToPILImage()

# loss_func = torch.nn.functional.mse_loss
# optimizer = torch.optim.Adam(ncp.parameters(), lr=0.001)

# # basic_agent = BasicAgent(vehicle=vehicle)
# trainer = Trainer(ncp, loss_func, optimizer)

while True:
    tmp += 1
    # if vehicle.get_location().distance(dest) < 0.5:
    #     print(f"Destination reached")
    #     break
    if (vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints / 4) or \
        (vehicle.get_location().distance(waypoint.transform.location) >= dist_between_waypoints * 2 and \
         vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints * 3):
        print(f'Waypoint {idx} was reached')
        if idx + 1 >= len(waypoints):
            print("The target has been reached, stopping the simulation")
            break
        waypoint = waypoints[idx + 1]
        idx += 1

    # control, movement, raw_data, out_tensor = agent.run_step()
    control, _, raw_data= agent.run_step(dump_data=True)
    vehicle.apply_control(control)
    if agent.simulator.image_frame is not None:
        with open(f'out/{simulator.world_name}/data.txt', 'a+') as f:
            f.write(f'{agent.simulator.image_frame} : {control.steer}\n')

    
    if agent.done():
        if dest_idx < waypoint_num - 1:
            dest_idx += 10
            dest_idx = min(dest_idx, waypoint_num - 1)
            print(f'Intermediate destination reached. Moving to waypoint {dest_idx}')
            agent.is_done = False
            agent.set_destination(waypoints[dest_idx].transform.location)
            continue

        print("The target has been reached, stopping the simulation")
        break
vehicle.apply_control(carla.VehicleControl(throttle = 0.0, brake=1.0, steer = 0.0))

In [ ]:
simulator.destroy_all()

# training

# test

In [1]:
import carla
import numpy as np
from src.simulator import Simulator
from src.agent import NCPAgent
from src.model import Model, DrivingModelModule, CustomDataset

from random import uniform, random

import torchvision
from IPython.display import display
import sys
import os
import torch
sys.path.append("CARLA_SIM/PythonAPI/carla/")
from agents.navigation.basic_agent import BasicAgent


In [2]:
sequence_len = 5
simulator = Simulator(world_name='Town04_opt',
                      debug=False,
                      dump_data=False,
                      sequence_len=sequence_len)

In [4]:
random_rotation = 3 * random() - 1.5
mapping = {
    'Town01_opt': {
        'loc': carla.Location(x=176.589493, y=133.239151, z=0.300000), # 15 spawn point
        'rotation': carla.Rotation(0, 0 + random_rotation, 0)
    },
    'Town02_opt': {
        'loc': carla.Location(x=193.779999, y=142.190002, z=0.500000), # 15 spawn point
        'rotation': carla.Rotation(0, -90 + random_rotation, 0)
    },
    'Town03_opt': {
        'loc': carla.Location(x=-88.710991, y=-119.565231, z=0.275307), # 1 spawn point
        'rotation': carla.Rotation(0, 90 + random_rotation, 0)
    },
    'Town04_opt': {
        'loc': carla.Location(405.320374, -48.450779, 0.281942), #222 spawn point
        'rotation': carla.Rotation(0, -90 + random_rotation, 0)
    },
}
print(random_rotation)

-1.484836432611937


In [5]:
simulator.spawn_car_with_camera(
    rel_coordinates=carla.Location(x=1.2, z=1.9), # camera coords
    vehicle_coordinates=mapping[simulator.world_name]['loc'],
    vehicle_rotation=mapping[simulator.world_name]['rotation'],
    image_param=(640,640)
)
# vehicle = simulator.get_vehicle()
# print(f'Transforms : {vehicle.get_transform()}')

output_size = 4
# units = 32
units = 64
# units = 128
# units = 256
# units = 512
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [6]:
checkpoint_name = f"model/max_ep_10_units_{units}_seq_{sequence_len}_lr_0.0001/epoch_epoch=09_val_loss=2.89605.ckpt"

In [7]:
base_model = Model(output_size, units)
ncp = DrivingModelModule.load_from_checkpoint(checkpoint_name,
    model=base_model,
    loss_func=torch.nn.L1Loss(),
    optimizer_cls=torch.optim.Adam,
    optimizer_kwargs={'lr': 0.0001},
    stb_weights=[1.0, 0.0, 0.0]
)
ncp.eval()
agent = NCPAgent(simulator, ncp, target_speed=10)

alloc!
Camera stream started


In [8]:
vehicle = simulator.get_vehicle()
next_waypoint  = [simulator.world.get_map().get_waypoint(vehicle.get_location(),
                                                    project_to_road=True,
                                                    lane_type=(carla.LaneType.Driving))]
waypoints = []
dist_between_waypoints = 15
waypoint_num = 10
# waypoint_num = 350
# waypoint_num = 100
# waypoint_num = 10
for _ in range(waypoint_num):
    waypoints.append(next_waypoint[-1])
    # simulator.world.get_spectator().set_transform(next_waypoint[-1].transform)
    next_waypoint = next_waypoint[-1].next(dist_between_waypoints)

dest_idx = 2
dest = waypoints[dest_idx].transform.location
agent.set_destination(dest)
agent.set_target_speed(10)
agent.ignore_traffic_lights(active=True)
agent.ignore_stop_signs(active=True)

In [9]:
# # os.rmdir('./out/')

idx = 1
tmp = 0
waypoint = waypoints[idx]

loc_and_waypoint_map = []

while True:
    tmp += 1
    if (vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints / 4) or \
        (vehicle.get_location().distance(waypoint.transform.location) >= dist_between_waypoints * 2 and \
         vehicle.get_location().distance(waypoint.transform.location) <= dist_between_waypoints * 3):
        print(f'Waypoint {idx} was reached')
        if idx + 1 >= len(waypoints):
            print("The target has been reached, stopping the simulation")
            break
        waypoint = waypoints[idx + 1]
        idx += 1

    # control, movement, raw_data, out_tensor = agent.run_step()
    control, model_control, raw_data= agent.run_step(dump_data=False)
    model_steer = 2 * (model_control[0][-1][0].item() + 70) / 140 - 1 # to [-1;1]
    print(control.steer, model_steer)
    vehicle.apply_control(carla.VehicleControl(throttle=control.throttle,
                                               brake=control.brake,
                                               steer=model_steer))
    
    loc_and_waypoint_map.append((vehicle.get_location(), idx))
    
    if agent.done():
        if dest_idx < waypoint_num - 1:
            dest_idx += 10
            dest_idx = min(dest_idx, waypoint_num - 1)
            print(f'Intermediate destination reached. Moving to waypoint {dest_idx}')
            agent.is_done = False
            agent.set_destination(waypoints[dest_idx].transform.location)
            continue

        print("The target has been reached, stopping the simulation")
        break
vehicle.apply_control(carla.VehicleControl(throttle = 0.0, brake=1.0, steer = 0.0))

150 5
0.06570109724998474 -7.275126076122795e-06
150 5
0.06586955487728119 -4.821955891576302e-06
150 5
0.06595379114151001 -4.772017044629706e-06
150 5
0.06603801995515823 -4.772030349320389e-06
150 5
0.06612225621938705 -4.77205030624539e-06
150 5
0.06620648503303528 -4.77205030624539e-06
150 5
0.0662907212972641 -4.77205030624539e-06
150 5
0.06637495011091232 -4.77205030624539e-06
150 5
0.06645918637514114 -4.77205030624539e-06
150 5
0.06654341518878937 -4.77205030624539e-06
150 5
0.06654341518878937 -4.77205030624539e-06
150 5
0.06654341518878937 -4.77205030624539e-06
150 5
0.06654341518878937 -4.77205030624539e-06
150 5
0.06654341518878937 -4.77205030624539e-06
150 5
0.06665767729282379 -4.77205030624539e-06
150 5
0.06714392453432083 -4.77205030624539e-06
150 5
0.06812766194343567 -4.77205030624539e-06
150 5
0.06813094764947891 -4.77205030624539e-06
150 5
0.06863676756620407 -4.77205030624539e-06
150 5
0.06838071346282959 -4.77205030624539e-06
150 5
0.06999377906322479 -4.77205030

KeyboardInterrupt: 

In [11]:
b = agent.sensors_data_storage.get_sensor_data('camera_front')
b[-10][0].shape

torch.Size([3, 640, 640])

In [15]:
from torchvision import transforms
from PIL import Image
import torch

to_pil = transforms.ToPILImage()
pil_image = to_pil(b[24][0])
pil_image.save('test1.jpg')

pil_image = to_pil(b[0][0])
pil_image.save('test2.jpg')


In [ ]:
def closest_point_on_segment_2d(current_point, line_a, line_b):
    # Convert to 2D numpy arrays
    ap = np.array([current_point.x - line_a.x, current_point.y - line_a.y])
    ab = np.array([line_b.x - line_a.x, line_b.y - line_a.y])
    ab_norm_sq = np.dot(ab, ab)
    if ab_norm_sq == 0:
        return carla.Location(x=line_a.x, y=line_a.y, z=0.0)
    # Project and clamp
    t = np.clip(np.dot(ap, ab) / ab_norm_sq, 0, 1)
    closest_x = line_a.x + ab[0] * t
    closest_y = line_a.y + ab[1] * t
    return carla.Location(x=closest_x, y=closest_y, z=0.0)

In [10]:
simulator.destroy_all()

Camera stream stopped


- l1 loss instead of mse
- sum l1 + l2?
- Training framework (pt lightning, catalyst)
- Save gt + loss. If gt 0 and loss 0 - irrelevant | gt 0 loss >> 0 -> relevant data

1. upscale image
2. agent logging
3. check if interactive can be added
4. Метрики отражающие ситуацию (насколько хорошо агент справляется с задачей)
   - скорость смещения от края в центру
   - средний угол отклонения от гт
   - отношение пересечений разметки к пройденной дист

In [ ]:
from src.model import CustomDataset
from PIL import Image
import torchvision
import matplotlib.pyplot as plt

datasets = [
    {'annotations_file': 'out/Town01_opt/data.txt', 'img_dir': 'out/Town01_opt/'},
    {'annotations_file': 'out/Town02_opt/data.txt', 'img_dir': 'out/Town02_opt/'},
    {'annotations_file': 'out/Town03_opt/data.txt', 'img_dir': 'out/Town03_opt/'},
    {'annotations_file': 'out/Town04_opt/data.txt', 'img_dir': 'out/Town04_opt/'},
]

dataset = CustomDataset(
    datasets=datasets,
    sequence_length=10
)

images = dataset[140][0].squeeze()
imgs = []
for img in images:
    # print(img.max(), img.min())
    _img = torchvision.transforms.functional.to_pil_image(img)
    imgs.append(_img)
dataset[140][1]

In [ ]:
import numpy as np
# arr = np.array(_img)
# arr.max(), arr.min()
x = torchvision.transforms.functional.pil_to_tensor(_img)

In [ ]:
x.max(), x.min()

In [ ]:
dataset[140][1]
# dataset[140][1]

In [ ]:
from src.model import CustomDataset

datasets = [
    {'annotations_file': 'out/Town01_opt/data.txt', 'img_dir': 'out/Town01_opt/'},
    {'annotations_file': 'out/Town02_opt/data.txt', 'img_dir': 'out/Town02_opt/'},
    {'annotations_file': 'out/Town03_opt/data.txt', 'img_dir': 'out/Town03_opt/'},
    {'annotations_file': 'out/Town04_opt/data.txt', 'img_dir': 'out/Town04_opt/'},
]

dataset = CustomDataset(
    datasets=datasets,
    sequence_length=10
)

steer_angles_train = dataset.data_groups[1]['steer_angles']
steer_angles_train.extend(dataset.data_groups[2]['steer_angles'])
steer_angles_train.extend(dataset.data_groups[3]['steer_angles'])

steer_angles_test = dataset.data_groups[0]['steer_angles']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame({'Steering angle':steer_angles_train})
plt.figure(figsize=(15,12))
ax = df['Steering angle'].hist(bins=100)
ax.set_xlabel("Steering angle")
ax.set_ylabel("Number of samples")

plt.xticks([x for x in range(-60, 60, 2)])

ax

In [ ]:
df = pd.DataFrame({'Steering angle':steer_angles_test})
ax = df['Steering angle'].hist()
ax.set_xlabel("Steering angle")
ax.set_ylabel("Number of samples")